# CS 4622 - Machine Learning - Lab 01

## Feature Engineering

The task in this lab was to develop a model to predict 4 label outputs, by applying feature engineering techniques.
We were provided with 2 datasets : "train.csv" and "valid.csv" for this lab.

Given below is the general outline of how the implementation has been done in this notebook:
<ol>
<li> Read given data into pandas dataframe </li> 
<li> Fill missing values </li> 
<li> For all 4 labels, </li> 
    <ol>
    <li> Train a SVM model for all 256 features </li> 
    <li> Get accuracy for the trained model </li> 
    <li> Perform feature selection and train a new model for the reduced number of features </li> 
    <li> Get accuracy scores for the new model and evaluate the performance </li> 
    </ol>
</ol>
    
For 'label_4' an additional step of preprocessing was done to oversample the data and ensure an equal distribution.

In [1]:
# Import necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import RandomOverSampler

In [2]:
# Read csv data
train = pd.read_csv("train.csv")
valid = pd.read_csv("valid.csv")

In [3]:
# Display information about the train and valid datasets
train_original = train.copy()
valid_original = valid.copy()

print(train.shape)
print(valid.shape)

(28520, 260)
(750, 260)


In [4]:
# Display the first few rows of 'train' dataframe
train.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_251,feature_252,feature_253,feature_254,feature_255,feature_256,label_1,label_2,label_3,label_4
0,-1.928826,-0.291540,1.105909,2.070652,0.657838,-0.670940,-0.968238,0.049916,1.111149,-1.834592,...,-0.691538,-1.152522,0.441157,-1.494434,-0.344974,-0.395641,45,NaN,1,6
1,-1.709277,-1.008998,0.956611,4.313823,-0.669455,1.928765,0.162550,0.152173,2.356505,-2.898549,...,-0.668235,-0.532695,0.151163,-1.876885,1.651534,-2.706490,45,NaN,1,6
2,-1.906183,-0.357562,-0.682627,4.651838,-0.884940,1.159512,1.354481,-1.432248,2.160144,-3.175735,...,-0.026728,0.440408,2.745906,-0.680754,-0.643588,-2.587203,45,NaN,1,6
3,0.697872,-2.218567,-0.572214,1.192062,0.329554,0.741364,0.962354,0.993512,2.896854,-1.600380,...,0.709032,0.812106,1.055266,-0.915258,-0.924856,-1.318964,45,NaN,1,6
4,-1.976895,-0.496308,0.368102,3.739787,-0.132058,1.234980,-0.501003,0.881554,1.630941,-3.853953,...,-0.599700,-1.024829,1.869753,-2.498570,3.268253,-1.356606,45,NaN,1,6


In [5]:
# Display the first few rows of 'valid' dataframe
valid.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_251,feature_252,feature_253,feature_254,feature_255,feature_256,label_1,label_2,label_3,label_4
0,-1.153148,-1.038098,1.419872,2.734152,1.154604,-1.086937,-0.516225,-1.370325,2.865359,-1.879877,...,-0.719046,-1.248530,0.144460,-3.240056,0.052614,0.083108,45,NaN,1,6
1,-1.377524,-1.018393,1.102352,2.849025,0.440302,-1.149039,-0.789796,-2.258196,1.264268,-2.123730,...,-1.383339,-0.787736,1.044895,-2.289637,0.199752,-0.712154,45,NaN,1,6
2,0.889574,-2.743300,-0.320194,3.047766,-0.923335,1.741686,-0.615148,0.756482,2.074775,-1.433126,...,1.178322,0.035333,0.857712,-1.928684,0.639870,-0.268576,45,NaN,1,6
3,-1.527213,-1.133121,0.385927,3.129767,0.229020,1.373105,0.919284,-0.755558,1.086973,-2.440614,...,-0.664176,0.805059,0.975368,-2.700269,1.523236,-1.259052,45,NaN,1,6
4,0.948176,-0.750248,0.008329,1.675338,1.941155,-0.783623,-0.485584,-0.261882,2.875204,-1.473030,...,-0.665200,-0.428060,-0.393100,-1.854657,2.207063,-0.342725,45,NaN,1,6


In [6]:
# Find out the number of rows with missing values for each label
def check_missing_values(label):
    print("Missing values in {label}: {val}".format(label=label, val=train[label].isna().sum()))

for label in ['label_1', 'label_2', 'label_3', 'label_4']:
    check_missing_values(label)

Missing values in label_1: 0
Missing values in label_2: 480
Missing values in label_3: 0
Missing values in label_4: 0


In [7]:
# Handling missing values in Label 2
train['label_2'].fillna(train['label_2'].mean().round(), inplace=True)
valid['label_2'].fillna(valid['label_2'].mean().round(), inplace=True)

train = train.astype({'label_2':'int'})
valid = valid.astype({'label_2':'int'})

# Confirm that the values have been filled
train.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_251,feature_252,feature_253,feature_254,feature_255,feature_256,label_1,label_2,label_3,label_4
0,-1.928826,-0.291540,1.105909,2.070652,0.657838,-0.670940,-0.968238,0.049916,1.111149,-1.834592,...,-0.691538,-1.152522,0.441157,-1.494434,-0.344974,-0.395641,45,28,1,6
1,-1.709277,-1.008998,0.956611,4.313823,-0.669455,1.928765,0.162550,0.152173,2.356505,-2.898549,...,-0.668235,-0.532695,0.151163,-1.876885,1.651534,-2.706490,45,28,1,6
2,-1.906183,-0.357562,-0.682627,4.651838,-0.884940,1.159512,1.354481,-1.432248,2.160144,-3.175735,...,-0.026728,0.440408,2.745906,-0.680754,-0.643588,-2.587203,45,28,1,6
3,0.697872,-2.218567,-0.572214,1.192062,0.329554,0.741364,0.962354,0.993512,2.896854,-1.600380,...,0.709032,0.812106,1.055266,-0.915258,-0.924856,-1.318964,45,28,1,6
4,-1.976895,-0.496308,0.368102,3.739787,-0.132058,1.234980,-0.501003,0.881554,1.630941,-3.853953,...,-0.599700,-1.024829,1.869753,-2.498570,3.268253,-1.356606,45,28,1,6


In [8]:
# Confirm whether the missing values are all completed now

for label in ['label_1', 'label_2', 'label_3', 'label_4']:
    check_missing_values(label)

Missing values in label_1: 0
Missing values in label_2: 0
Missing values in label_3: 0
Missing values in label_4: 0


In [9]:
# Separate features and target labels
X_train = train.iloc[:, :256]
y_train = train[['label_1', 'label_2', 'label_3', 'label_4']]

X_valid = valid.iloc[:, :256]
y_valid = valid[['label_1', 'label_2', 'label_3', 'label_4']]

# Check the dimensions to confirm
print(X_train.shape, X_valid.shape)
print(y_train.shape, y_valid.shape)

(28520, 256) (750, 256)
(28520, 4) (750, 4)


## 1. Modelling Speaker ID

Speaker ID is labeled as 'label_1' in the datasets. There were no missing values for this label. This can be considered a categorical variable. 

First, a model will be developed for all 256 features. I have used a SVM classifier for this model.

In [10]:
svm_model_1 = SVC(kernel='linear', C=1.0, random_state=42)
svm_model_1.fit(X_train, y_train['label_1'])

SVC(kernel='linear', random_state=42)

In [11]:
# Evaluate the model on valid dataset
from sklearn.metrics import accuracy_score

y_pred_1 = svm_model_1.predict(X_valid)
accuracy = accuracy_score(y_valid['label_1'], y_pred_1)
print("Accuracy:", accuracy)

Accuracy: 0.992


Now, some feature engineering techniques will be applied to select the most impactful features from the 256 features. 

In [12]:
# Feature Selection using SelectKBest and ANOVA F-value test
# k = num of features to select

selector1 = SelectKBest(score_func=f_classif, k=100)
X_train_selected = selector1.fit_transform(X_train, y_train['label_1'])
X_valid_selected = selector1.transform(X_valid)

In [13]:
# Print the dimensions of X_train_selected
X_train_selected.shape

(28520, 100)

In [14]:
# Retrain a new SVM Classifier for the selected feature set

svm_model_1_sel = SVC(kernel='linear', C=1.0, random_state=42)
svm_model_1_sel.fit(X_train_selected, y_train['label_1'])

SVC(kernel='linear', random_state=42)

In [15]:
# Evaluate the new model on valid dataset
y_pred_1_sel = svm_model_1_sel.predict(X_valid_selected)
accuracy = accuracy_score(y_valid['label_1'], y_pred_1_sel)
print("Accuracy:", accuracy)

Accuracy: 0.9773333333333334


Although the accuracy has dropped a little bit, it was possible to achieve a high accuracy rate with only 100 features. 

For label_1, <br>
Initial Model (trained with 256 features) : svm_model_1 <br>
Model after feature selection : svm_model_1_sel

## Modelling Speaker Age

Speaker age has been labeled as 'label_2'. The missing values were filled earlier with the mean age. 

In [16]:
svm_model_2 = SVC(kernel='linear', C=1.0, random_state=42)
svm_model_2.fit(X_train, y_train['label_2'])

SVC(kernel='linear', random_state=42)

In [17]:
y_pred_2 = svm_model_2.predict(X_valid)
accuracy = accuracy_score(y_valid['label_2'], y_pred_2)
print("Accuracy:", accuracy)

Accuracy: 0.9


In [18]:
# Feature Selection using SelectKBest and ANOVA F-value test
# k = num of features to select

selector2 = SelectKBest(score_func=f_classif, k=150)
X_train_selected = selector2.fit_transform(X_train, y_train['label_2'])
X_valid_selected = selector2.transform(X_valid)

In [19]:
# Retrain a new SVM Classifier for the selected feature set

svm_model_2_sel = SVC(kernel='linear', C=1.0, random_state=42)
svm_model_2_sel.fit(X_train_selected, y_train['label_2'])

SVC(kernel='linear', random_state=42)

In [20]:
# Evaluate the new model on valid dataset
y_pred_2_sel = svm_model_2_sel.predict(X_valid_selected)
accuracy = accuracy_score(y_valid['label_2'], y_pred_2_sel)
print("Accuracy:", accuracy)

Accuracy: 0.8653333333333333


For label_2, <br>
Initial model : svm_model_2 <br>
Model after feature selection : svm_model_2_sel

## Modelling Speaker Gender

Speaker gender is labeled as 'label_3'. This is a binary classification.

In [21]:
# Train an SVM model for all 256 features
svm_model_3 = SVC(kernel='linear', C=1.0, random_state=42)
svm_model_3.fit(X_train, y_train['label_3'])

# Evaluate with the valid dataset
y_pred_3 = svm_model_3.predict(X_valid)
accuracy = accuracy_score(y_valid['label_3'], y_pred_3)
print("Accuracy:", accuracy)

Accuracy: 0.9986666666666667


In [22]:
# Feature Selection using SelectKBest and ANOVA F-value test
# k = num of features to select
selector3 = SelectKBest(score_func=f_classif, k=50)
X_train_selected = selector3.fit_transform(X_train, y_train['label_3'])
X_valid_selected = selector3.transform(X_valid)

In [23]:
# Retrain a new SVM Classifier for the selected feature set
svm_model_3_sel = SVC(kernel='linear', C=1.0, random_state=42)
svm_model_3_sel.fit(X_train_selected, y_train['label_3'])

# Evaluate the new model on valid dataset
y_pred_3_sel = svm_model_3_sel.predict(X_valid_selected)
accuracy = accuracy_score(y_valid['label_3'], y_pred_3_sel)
print("Accuracy:", accuracy)

Accuracy: 0.9946666666666667


A very high accuracy has been achieved with 50 selected features.

For label_3, <br>
Initial model : svm_model_2 <br>
Model after feature selection : svm_model_2_sel

## Modelling Speaker Accent

Speaker accent is labelled as 'label_4'. The data in this column are not equally distributed. Thus, an oversampling technique has to be applied first before training a model. I have used RandomOverSampler() for this.

In [24]:
# Oversample the minority class using RandomOverSampler
oversampler = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train['label_4'])

For label_4, I developed a KNN model.

In [25]:
# From the square root rule, get the k value
k = int(X_train.shape[0] ** (1/4))
print(k)

12


In [26]:
# build the KNN classifier
knn_model = KNeighborsClassifier(n_neighbors=k)
knn_model.fit(X_train_resampled, y_train_resampled)

# Oversample the valid dataset
X_valid_resampled, y_valid_resampled = oversampler.fit_resample(X_valid, y_valid['label_4'])

y_pred_4 = knn_model.predict(X_valid_resampled)
accuracy = accuracy_score(y_valid_resampled, y_pred_4)
print("Accuracy:", accuracy)

Accuracy: 0.9981203007518797


In [27]:
# Feature Selection using SelectKBest and ANOVA F-value test

selector4 = SelectKBest(score_func=f_classif, k=100)
X_train_selected = selector4.fit_transform(X_train_resampled, y_train_resampled)
X_valid_selected = selector4.transform(X_valid_resampled)

In [28]:
# Retrain the model

knn_model_sel = KNeighborsClassifier(n_neighbors=k)
knn_model_sel.fit(X_train_selected, y_train_resampled)

y_pred_4_sel = knn_model_sel.predict(X_valid_selected)
accuracy = accuracy_score(y_valid_resampled, y_pred_4_sel)
print("Accuracy:", accuracy)

Accuracy: 0.9763694951664876


With only 100 features 97.6% accuracy has been achieved.

For label_4, <br>
Initial model : knn_model, <br>
Model after feature selection : knn_model_sel

## Testing the models

The accuracy of the above 4 models will be tested with a test dataset : "test.csv"

In [33]:
# Read test dataset file
test = pd.read_csv("test.csv")

In [34]:
X_test = test.iloc[:, :256]
print(X_test.shape)

(750, 256)


In [35]:
# Preprocess to input to the models
test_1 = selector1.transform(X_test)
test_2 = selector2.transform(X_test)
test_3 = selector3.transform(X_test)
test_4 = selector4.transform(X_test)

In [36]:
print(test_1.shape)
print(test_2.shape)
print(test_3.shape)
print(test_4.shape)

(750, 100)
(750, 150)
(750, 50)
(750, 100)


In [38]:
# Predict with the models before feature engineering 
y_test_1_bf = svm_model_1.predict(X_test)
y_test_2_bf = svm_model_2.predict(X_test)
y_test_3_bf = svm_model_3.predict(X_test)
y_test_4_bf = knn_model.predict(X_test)

In [39]:
# Predict with the models after feature engineering
y_test_1 = svm_model_1_sel.predict(test_1)
y_test_2 = svm_model_2_sel.predict(test_2)
y_test_3 = svm_model_3_sel.predict(test_3)
y_test_4 = knn_model_sel.predict(test_4)

In [40]:
# Write the transformed datasets into csv files

import csv

header = ["Predicted labels before feature engineering", "Predicted labels after feature engineering", "No of new features"]
for i in range(1, 257):
    header.append("new_feature_" + str(i))

with open("190547P_label_1.csv", "w", newline='') as f1:
    writer = csv.writer(f1)
    writer.writerow(header)
    for i in range(750):
        datarow = [y_test_1_bf[i], y_test_1[i], test_1.shape[1]] + test_1[i].tolist()
        writer.writerow(datarow)

with open("190547P_label_2.csv", "w", newline='') as f2:
    writer = csv.writer(f2)
    writer.writerow(header)
    for i in range(750):
        datarow = [y_test_2_bf[i], y_test_2[i], test_2.shape[1]] + test_2[i].tolist()
        writer.writerow(datarow)
        
with open("190547P_label_3.csv", "w", newline='') as f3:
    writer = csv.writer(f3)
    writer.writerow(header)
    for i in range(750):
        datarow = [y_test_3_bf[i], y_test_3[i], test_3.shape[1]] + test_3[i].tolist()
        writer.writerow(datarow)
        
with open("190547P_label_4.csv", "w", newline='') as f4:
    writer = csv.writer(f4)
    writer.writerow(header)
    for i in range(750):
        datarow = [y_test_4_bf[i], y_test_4[i], test_4.shape[1]] + test_4[i].tolist()
        writer.writerow(datarow)